<h3>Start State Randomizer<h3>

Randomly places the eight tiles in the 3 × 3 grid. Checks whether it is a solvable state using the
inversion test. This is a parity test for the state that counts the number of inversions of pairs
of tiles (i, j) when written linearly row-wise (excluding 0, the blank tile). An inversion is when
tile(i) > tile(j) where j > i. 

In [2]:
import random

def printgrid(a):
    for j in range(0,9):
        if a[j]=="0":print(" ",end=" ")
        else:print(a[j],end=" ")
        if((j+1)%3==0):print(" ")

class nodes:
    def __init__(self,parent,state,pos,he):  
        self.parent = parent   
        self.state = state
        self.pos = pos
        self.he = he
        if(parent==None):
            self.depth = 0
        else:
            self.depth = parent.depth + 1
        
class puzzle:
    def __init__(self):
        #self.curr = self.play()
        self.goal = "123456780"
        self.cur = ""
        
    def checklegal(self):
        CurrentState = self.cur
        inversion = 0
        for i in range(0,9):
            if  CurrentState[i]!="0":
                for j in range(i+1,9):
                    if CurrentState[i]>CurrentState[j] and CurrentState[j]!="0":
                        inversion+=1
        if(inversion%2==0):
            return True
        else:
            return False
    
    def StateGen(self):
        while(True):
            start = self.goal
            new = ""
            for i in range(0,9):
                m = random.randint(0,len(start)-1)
                temp = start[m]
                start = start[:m]+start[m+1:]
                new+=temp
            self.cur = new
            if self.checklegal():
                print("LEGAL START STATE IS\n")
                printgrid(new)
                break
p1 = puzzle()
p1.StateGen()

LEGAL START STATE IS

1 6 5  
7 8    
2 4 3  


<h3>Solver<h3>

1. For heruritic Number of tiles out of place. enter option A 

2. For heruritic Euclidean Distance enter option B 

3. For heruritic City Block Distance enter option C

4. For heruritic Tiles out of correct row and col enter option D

In [8]:
import random
import math

def printgrid(a):
    for j in range(0,9):
        if a[j]=="0":print(" ",end=" ")
        else:print(a[j],end=" ")
        if((j+1)%3==0):print(" ")

        
def minel(l):
    curmin = l[0].he + l[0].depth
    pos = 0
    for i in range(0,len(l)):
        if curmin>l[i].he+l[i].depth:
            curmin = l[i].he+l[i].depth
            pos = i
    return pos
                
def printPath(i):
    if i != None:
        printPath(i.parent)
        printgrid(i.state)
        print("fe = ",i.he+i.depth)
        print("")
            
class nodes:
    def __init__(self,parent,state,pos,he):  
        self.parent = parent   
        self.state = state
        self.pos = pos
        self.he = he
        if(parent==None):
            self.depth = 0
        else:
            self.depth = parent.depth + 1
        
class puzzle:
    def __init__(self):
        #self.curr = self.play()
        self.goal = "123456780"
        self.choice = input("Enter Heuristic choice: ")
        self.cur = ""
        self.StateGen()
    
    def isNotGoalNode(n):
        if(self.cur!=self.goal):
            return True
        return False
    
            
    def blankpos(self,ini):
        for i in range(0,9):
            if(ini[i]=='0'):
                return i
    
    def move(self,x):
        ini = self.cur.state
        k = self.blankpos(ini)
        if x==0 and k not in [0,1,2]:
            a = ini
            temp = a[k-3]
            return a[:k-3]+"0"+a[k-3+1:k]+temp+a[k+1:]
        elif x==1 and k not in [8,7,6]:
            a = ini
            temp = a[k+3]
            return a[:k]+temp+a[k+1:k+3]+"0"+a[k+3+1:]
        elif x==2 and k not in [0,3,6]:
            a = ini
            temp = a[k-1]
            return a[:k-1]+"0"+temp+a[k+1:]
        elif x==3 and k not in [2,5,8]:
            a = ini
            temp = a[k+1]
            return a[:k]+temp+"0"+a[k+2:]
        return 0
        
    def children(self):
        children = []
        for x in range (0,4):
            check = self.move(x)
            if(check!=0):
                he = self.selct_Heuristic(check)
                if check==self.goal: 
                    self.cur = nodes(self.cur,check,self.blankpos(check),he)
                    return 0
                if(self.cur.parent!=None):
                    if (check!=self.cur.parent.state): 
                        children.append(nodes(self.cur,check,self.blankpos(check),he))
                else: 
                    children.append(nodes(self.cur,check,self.blankpos(check),he))
        return children
    
    def checklegal(self):
        CurrentState = self.cur
        inversion = 0
        for i in range(0,9):
            if  CurrentState[i]!="0":
                for j in range(i+1,9):
                    if CurrentState[i]>CurrentState[j] and CurrentState[j]!="0":
                        inversion+=1
        if(inversion%2==0):
            return True
        else:
            return False
    
    def StateGen(self):
        while(True):
            start = self.goal
            new = ""
            for i in range(0,9):
                m = random.randint(0,len(start)-1)
                temp = start[m]
                start = start[:m]+start[m+1:]
                new+=temp
            self.cur = new
            if self.checklegal():
                printgrid(new)
                he = self.selct_Heuristic(new)
                self.cur = nodes(None,new,self.blankpos(new),he)
                break
                
    def heuristicA(self,n):
        g = self.goal
        output = 0
        for i in range(0,len(g)):
            if (g[i]!=n[i]):
                output+=1
        return output
    
    def heuristicB(self,n):
        g = self.goal
        output = 0
        for i in range(0,len(g)):
            for j in range(0,len(g)):
                if (g[i]==n[j]):
                    x1 = i % 3                    
                    y1 = i // 3
                    x2 = j % 3
                    y2 = j // 3
                    output += math.ceil(math.dist([x1, y1], [x2, y2]))
        return output
    
    def heuristicC(self,n):
        g = self.goal
        output = 0
        for i in range(0,len(g)):
            for j in range(0,len(g)):
                if (g[i]==n[j]):
                    x1 = i % 3                    
                    y1 = i // 3
                    x2 = j % 3
                    y2 = j // 3
                    output += abs(x1-x2)+abs(y1-y2)
        return output
    
    def heuristicD(self,n):
        g = self.goal                                   #Tiles out of row and col
        output = 0
        for i in range(0,len(g)):
            for j in range(0,len(g)):
                if (g[i]==n[j]):
                    x1 = i % 3                    
                    y1 = i // 3
                    x2 = j % 3
                    y2 = j // 3
                    if x1!=x2:output+=1
                    if y1!=y2:output+=1
        return output

    def selct_Heuristic(self,n):
        a = self.choice
        if a=="A":
            return self.heuristicA(n)
        elif a=="B":
            return self.heuristicB(n)
        elif a=="C":
            return self.heuristicC(n)
        elif a=="D":
            return self.heuristicD(n)
        elif a=="E":
            return 5*self.heuristicC(n)
            
    def Astar(self):
        t = 0
        visited = []
        notVisited = []
        notVisited.append(self.cur)
        while (notVisited!=[] and self.cur.he!=0):
            mincurrent =  self.cur.he + self.cur.depth
            pos = minel(notVisited)
            self.cur = notVisited.pop(pos)
            visited.append(self.cur)
            childnodes = self.children()
            if childnodes!=0:
                for i in childnodes:
                    flag=0
                    for j in range(0,len(visited)):
                        if visited[j].state==i.state:
                            flag=0
                            fe = visited[j].he + visited[j].depth
                            feChild = i.he+i.depth
                            if(feChild<fe):
                                flag=1
                                visited.pop(j)
                                notVisited.append(i)
                                break
                    if flag==0:
                        for j in range(0,len(notVisited)):
                            if notVisited[j].state==i.state:
                                flag=1
                                break
                        if flag==0:
                            notVisited.append(i)
            else:break
        print(" ")
        print("Found at Depth =",self.cur.depth)
        printPath(self.cur)
                                

p1 = puzzle()
j = p1.cur
# print(p1)
p1.Astar()
# p1.cur = j
# p1.choice = "C"
# p1.Astar()

Enter Heuristic choice: E
2 4 1  
3 8 5  
6   7  
 
Found at Depth = 43
2 4 1  
3 8 5  
6   7  
fe =  80

2 4 1  
3   5  
6 8 7  
fe =  81

2 4 1  
  3 5  
6 8 7  
fe =  82

2 4 1  
6 3 5  
  8 7  
fe =  73

2 4 1  
6 3 5  
8   7  
fe =  74

2 4 1  
6 3 5  
8 7    
fe =  65

2 4 1  
6 3    
8 7 5  
fe =  76

2 4 1  
6   3  
8 7 5  
fe =  77

2   1  
6 4 3  
8 7 5  
fe =  78

2 1    
6 4 3  
8 7 5  
fe =  69

2 1 3  
6 4    
8 7 5  
fe =  60

2 1 3  
6 4 5  
8 7    
fe =  51

2 1 3  
6 4 5  
8   7  
fe =  62

2 1 3  
6   5  
8 4 7  
fe =  73

2 1 3  
  6 5  
8 4 7  
fe =  74

2 1 3  
8 6 5  
  4 7  
fe =  75

2 1 3  
8 6 5  
4   7  
fe =  66

2 1 3  
8 6 5  
4 7    
fe =  57

2 1 3  
8 6    
4 7 5  
fe =  68

2 1 3  
8   6  
4 7 5  
fe =  69

2 1 3  
  8 6  
4 7 5  
fe =  70

  1 3  
2 8 6  
4 7 5  
fe =  81

1   3  
2 8 6  
4 7 5  
fe =  72

1 8 3  
2   6  
4 7 5  
fe =  73

1 8 3  
  2 6  
4 7 5  
fe =  74

1 8 3  
4 2 6  
  7 5  
fe =  65

1 8 3  
4 2 6  
7   5  
fe =  56

1 8 3  
4 

<h3> Testing Heuristics Nodes Expanded

5.  inadmissible heuristic of the form fe(n) = ge(n) + c×he(n) where he(n) is City Block Distance Option E 

In [13]:
import random
import math

class node:
    def __init__(self,parent,mats,pos):  
        self.parent = parent   
        self.mats = mats
        self.pos = pos
        if(parent==None):
            self.depth = 0
        else:
            self.depth = parent.depth + 1
                
def F(count):
    print("E is inadmissable heuristic")
    e = 0
    c = 0
    cou = 0
    cost = []
    expanded = [0,0,0,0,0]
    ia = 0
    while count>cou:
        p1 = puzzle()
        start = p1.cur
        l = ["E","A","B","C","D"]
        for j in range(0,len(l)):
            p1.cur = start
            p1.choice = l[j]
            temp = p1.Astar()
            expanded[j]+=temp[1]
            if l[j]=="E":
                if e==0:
                    cost.append(temp[0])
                else:
                    cost[0] = temp[0]
                e+=1
            if l[j]=="B":
                if c==0:
                    cost.append(temp[0]/cost[0])
                else:
                    cost[1] = (temp[0]/cost[0])+cost[1]
                c+=1
        cou+=1
    print("\n\nThe average ratio of the cost of the optimal path to that found by the inadmissible heuristic(E) is: ",cost[1]/count)
    for i in range(0,len(l)):
        print("Avg nodes expanded in",l[i],"heuristic is:",expanded[i]/count)
            
def printgrid(a):
    for j in range(0,9):
        if a[j]=="0":print(" ",end=" ")
        else:print(a[j],end=" ")
        if((j+1)%3==0):print(" ")

        
def minel(l):
    curmin = l[0].he + l[0].depth
    pos = 0
    for i in range(0,len(l)):
        if curmin>l[i].he+l[i].depth:
            curmin = l[i].he+l[i].depth
            pos = i
    return pos
                
def printPath(i):
    if i != None:
        printPath(i.parent)
        printgrid(i.state)
        print("")
            
class nodes:
    def __init__(self,parent,state,pos,he):  
        self.parent = parent   
        self.state = state
        self.pos = pos
        self.he = he
        if(parent==None):
            self.depth = 0
        else:
            self.depth = parent.depth + 1
        
class puzzle:
    def __init__(self):
        #self.curr = self.play()
        self.goal = "123456780"
        self.choice = "E"
        self.cur = ""
        self.StateGen()
    
            
    def blankpos(self,ini):
        for i in range(0,9):
            if(ini[i]=='0'):
                return i
    
    def move(self,x):
        ini = self.cur.state
        k = self.blankpos(ini)
        if x==0 and k not in [0,1,2]:
            a = ini
            temp = a[k-3]
            return a[:k-3]+"0"+a[k-3+1:k]+temp+a[k+1:]
        elif x==1 and k not in [8,7,6]:
            a = ini
            temp = a[k+3]
            return a[:k]+temp+a[k+1:k+3]+"0"+a[k+3+1:]
        elif x==2 and k not in [0,3,6]:
            a = ini
            temp = a[k-1]
            return a[:k-1]+"0"+temp+a[k+1:]
        elif x==3 and k not in [2,5,8]:
            a = ini
            temp = a[k+1]
            return a[:k]+temp+"0"+a[k+2:]
        return 0
        
    def children(self):
        children = []
        for x in range (0,4):
            check = self.move(x)
            if(check!=0):
                he = self.selct_Heuristic(check)
                if check==self.goal: 
                    self.cur = nodes(self.cur,check,self.blankpos(check),he)
                    return 0
                if(self.cur.parent!=None):
                    if (check!=self.cur.parent.state): 
                        children.append(nodes(self.cur,check,self.blankpos(check),he))
                else: 
                    children.append(nodes(self.cur,check,self.blankpos(check),he))
        return children
    
    def checklegal(self):
        CurrentState = self.cur
        inversion = 0
        for i in range(0,9):
            if  CurrentState[i]!="0":
                for j in range(i+1,9):
                    if CurrentState[i]>CurrentState[j] and CurrentState[j]!="0":
                        inversion+=1
        if(inversion%2==0):
            return True
        else:
            return False
    
    def StateGen(self):
        while(True):
            start = self.goal
            new = ""
            for i in range(0,9):
                m = random.randint(0,len(start)-1)
                temp = start[m]
                start = start[:m]+start[m+1:]
                new+=temp
            self.cur = new
            if self.checklegal():
                print("")
                printgrid(new)
                he = self.selct_Heuristic(new)
                self.cur = nodes(None,new,self.blankpos(new),he)
                break
                
    def heuristicA(self,n):
        g = self.goal
        output = 0
        for i in range(0,len(g)):
            if (g[i]!=n[i]):
                output+=1
        return output
    
    def heuristicB(self,n):
        g = self.goal
        output = 0
        for i in range(0,len(g)):
            for j in range(0,len(g)):
                if (g[i]==n[j]):
                    x1 = i % 3                    
                    y1 = i // 3
                    x2 = j % 3
                    y2 = j // 3
                    output += math.ceil(math.dist([x1, y1], [x2, y2]))
        return output
    
    def heuristicC(self,n):
        g = self.goal
        output = 0
        for i in range(0,len(g)):
            for j in range(0,len(g)):
                if (g[i]==n[j]):
                    x1 = i % 3                    
                    y1 = i // 3
                    x2 = j % 3
                    y2 = j // 3
                    output += abs(x1-x2)+abs(y1-y2)
        return output
    
    def heuristicD(self,n):
        g = self.goal                                   #Tiles out of row and col
        output = 0
        for i in range(0,len(g)):
            for j in range(0,len(g)):
                if (g[i]==n[j]):
                    x1 = i % 3                    
                    y1 = i // 3
                    x2 = j % 3
                    y2 = j // 3
                    if x1!=x2:output+=1
                    if y1!=y2:output+=1
        return output
    
    def selct_Heuristic(self,n):
        a = self.choice
        if a=="A":
            return self.heuristicA(n)
        elif a=="B":
            return self.heuristicB(n)
        elif a=="C":
            return self.heuristicC(n)
        elif a=="D":
            return self.heuristicD(n)
        elif a=="E":
            return 3*self.heuristicC(n)
#         elif a==" ":
#             return 0
            
    def Astar(self):
        t = 0
        visited = []
        notVisited = []
        notVisited.append(self.cur)
        while (notVisited!=[] and self.cur.he!=0):
            t+=1
            mincurrent =  self.cur.he + self.cur.depth
            pos = minel(notVisited)
            self.cur = notVisited.pop(pos)
            visited.append(self.cur)
            childnodes = self.children()
            if childnodes!=0:
                for i in childnodes:
                    flag=0
                    for j in range(0,len(visited)):
                        if visited[j].state==i.state:
                            flag=0
                            fe = visited[j].he + visited[j].depth
                            feChild = i.he+i.depth
                            if(feChild<fe):
                                flag=1
                                visited.pop(j)
                                notVisited.append(i)
                                break
                    if flag==0:
                        for j in range(0,len(notVisited)):
                            if notVisited[j].state==i.state:
                                flag=1
                                break
                        if flag==0:
                            notVisited.append(i)
            else:break
        print(" ")
        print("Number of nodes expanded in ",self.choice," heuristic is = ",t)
        #print("COST: ",self.cur.depth)
        return [self.cur.depth,t]

x = int(input("Enter No of testcases:"))
F(x)

Enter No of testcases:3
E is inadmissable heuristic

  1 5  
4 6 3  
7 8 2  
 
Number of nodes expanded in  E  heuristic is =  63
 
Number of nodes expanded in  A  heuristic is =  742
 
Number of nodes expanded in  B  heuristic is =  213
 
Number of nodes expanded in  C  heuristic is =  207
 
Number of nodes expanded in  D  heuristic is =  335

8   5  
2 6 3  
1 4 7  
 
Number of nodes expanded in  E  heuristic is =  128
 
Number of nodes expanded in  A  heuristic is =  2631
 
Number of nodes expanded in  B  heuristic is =  319
 
Number of nodes expanded in  C  heuristic is =  358
 
Number of nodes expanded in  D  heuristic is =  862

2 3 4  
8 7 1  
5 6    
 
Number of nodes expanded in  E  heuristic is =  493
 
Number of nodes expanded in  A  heuristic is =  10116
 
Number of nodes expanded in  B  heuristic is =  1319
 
Number of nodes expanded in  C  heuristic is =  1293
 
Number of nodes expanded in  D  heuristic is =  3175


The average ratio of the cost of the optimal path to tha

As we can see that while the inadmissable heuristic does expand the least amount of nodes on average it returns a path that is non optimal on average 

Hence while it may be cost optimal in node expansions that is it is the quickest algorithm it almost always returns a non optimal path with a greater cost(cost here reffering to the number of moves to reach goal state).